In [52]:
%load_ext tensorboard

import datetime
import os
import numpy as np 
import tensorflow as tf 
from tensorflow import keras as kr
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.datasets import mnist 
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = mnist.load_data()

!rm -rf ./logs/

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [53]:
print('MNIST Dataset Shape:')
batch_size = 64 
buffer_size = 10000
steps_per_epoch = int(np.ceil(60000) / float(batch_size))
epochs = 10
num_classes = 10

X_train = X_train_orig.astype(np.float32)/255
X_train = np.expand_dims(X_train, -1)
X_test = X_test_orig.astype(np.float32)/255
X_test = np.expand_dims(X_test, -1)
Y_train = tf.one_hot(Y_train_orig, num_classes)
Y_test = tf.one_hot(Y_test_orig, num_classes)

print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

MNIST Dataset Shape:
X_train: (60000, 28, 28, 1)
Y_train: (60000, 10)
X_test:  (10000, 28, 28, 1)
Y_test:  (10000, 10)


In [54]:
def Conv_Model(inputs):
    X_input = kr.Input(inputs)
    X = layers.Conv2D(32, (3,3), activation = 'relu', padding = 'valid')(X_input)
    X = layers.MaxPooling2D(pool_size=(2,2))(X)
    X = layers.Conv2D(64, (3,3), activation = 'relu')(X)
    X = layers.MaxPooling2D(pool_size=(2,2))(X)
    X = layers.Flatten()(X)
    X = layers.Dense(512, activation = 'relu')(X)
    X = layers.Dropout(0.5)(X)
    X = layers.Dense(num_classes, activation='softmax', name = 'X_preds')(X)

    model = kr.models.Model(inputs = X_input, outputs = X, 
            name = "Mnist_Conv_Model")

    return model

In [55]:
Conv_Model = Conv_Model(X_train.shape[1:])
Conv_Model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ["accuracy"])
Conv_Model.fit(x = X_train, y = Y_train, epochs = 5, batch_size = 64, callbacks=[tensorboard_callback])

Epoch 1/5
938/938 [==============================] - 22s 23ms/step - loss: 0.1703 - accuracy: 0.9475
Epoch 2/5
938/938 [==============================] - 21s 22ms/step - loss: 0.0532 - accuracy: 0.9835
Epoch 3/5
938/938 [==============================] - 21s 22ms/step - loss: 0.0390 - accuracy: 0.9880
Epoch 4/5
938/938 [==============================] - 21s 22ms/step - loss: 0.0299 - accuracy: 0.9906
Epoch 5/5
938/938 [==============================] - 20s 21ms/step - loss: 0.0258 - accuracy: 0.9915


In [56]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 17596), started 0:02:07 ago. (Use '!kill 17596' to kill it.)

In [ ]:
predictions = Conv_Model.evaluate(x = X_test, y = Y_test)
print ("Loss = " + str(predictions[0]))
print ("Test Accuracy = " + str(predictions[1]))

In [9]:
Conv_Model.summary()

Model: "Mnist_Conv_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 512)          

In [10]:
import matplotlib as plt
%matplotlib inline

In [11]:
def Conv_Model_2(inputs):
    X_input = kr.Input(inputs)
    init = kr.initializers.GlorotNormal(seed=None)
    X = layers.Conv2D(32, (6,6), 
                    activation = 'relu', 
                    padding = 'same',                                                                     kernel_initializer = init)(X_input)
    X = layers.MaxPooling2D(pool_size=(2,2))(X)
    X = layers.Conv2D(64, (3,3), 
                    activation = 'relu', 
                    padding = 'same',
                    kernel_initializer = init)(X)
    X = layers.MaxPooling2D(pool_size=(2,2))(X)
    X = layers.Conv2D(64, (3,3), 
                    activation = 'relu', 
                    padding = 'same',
                    kernel_initializer = init)(X)
    X = layers.MaxPooling2D(pool_size=(2,2))(X)
    X = layers.Flatten()(X)
    X = layers.Dense(512, activation = 'relu')(X)
    X = layers.Dropout(0.5)(X)
    X = layers.Dense(num_classes, activation='softmax', name = 'X_preds')(X)

    model = kr.models.Model(inputs = X_input, outputs = X, 
            name = "Mnist_Conv_Model")

    return model

In [12]:
Conv_Model_2 = Conv_Model_2(X_train.shape[1:])
Conv_Model_2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ["accuracy"] )

In [94]:
Conv_Model_2.fit(x = X_train, y = Y_train, epochs = 5, batch_size = 16)

Epoch 1/5
 522/3750 [===>..........................] - ETA: 47s - loss: 0.1973 - accuracy: 0.9388

In [61]:
predictions = Conv_Model_2.evaluate(x = X_test, y = Y_test)
print ("Loss = " + str(predictions[0]))
print ("Test Accuracy = " + str(predictions[1]))

313/313 [==============================] - 2s 6ms/step - loss: 0.0410 - accuracy: 0.9881
Loss = 0.04103730246424675
Test Accuracy = 0.988099992275238


In [90]:
Conv_Model_2.summary()

Model: "Mnist_Conv_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 7, 7, 64)          36928     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 3, 3, 64)     